# V2 Notebook 3: Advanced Optimization with Genetic Algorithms

**Project:** `RobustMPC-Pharma` (V2)
**Goal:** Replace the V1 controller's inefficient exhaustive search with a scalable, intelligent optimization engine. We will use a **Genetic Algorithm (GA)** to efficiently search a massive space of potential control plans, enabling more sophisticated and effective control strategies.

### Table of Contents
1. [Theory: The Limits of Brute Force and the Power of Evolution](#1.-Theory:-The-Limits-of-Brute-Force-and-the-Power-of-Evolution)
2. [Introduction to Genetic Algorithms for Control Problems](#2.-Introduction-to-Genetic-Algorithms-for-Control-Problems)
3. [Building the `GeneticOptimizer` Class with DEAP](#3.-Building-the-GeneticOptimizer-Class-with-DEAP)
4. [Defining a Richer Action Space](#4.-Defining-a-Richer-Action-Space)
5. [Standalone Test of the Genetic Optimizer](#5.-Standalone-Test-of-the-Genetic-Optimizer)

--- 
## 1. Theory: The Limits of Brute Force and the Power of Evolution

Our V1 MPC controller used an **exhaustive search** over a discretized lattice. This approach has two critical flaws:

1.  **It is not scalable.** The number of possible plans grows exponentially with the number of control variables and discretization steps. For 3 CPPs with 3 discretization steps each, we had `3^3 = 27` candidates. For 5 CPPs with 5 steps, we would have `5^5 = 3,125` candidates. This quickly becomes computationally impossible to evaluate in real-time.
2.  **It is not expressive.** To keep the search space small, we had to assume a very simple control plan: 'make one change and hold it constant'. This prevents the controller from finding better, more dynamic solutions like gradually ramping a parameter.

We need a smarter way to search. Instead of checking every single possibility, we can use an **evolutionary algorithm** that intelligently explores the search space to find near-optimal solutions quickly.

--- 
## 2. Introduction to Genetic Algorithms for Control Problems

A Genetic Algorithm (GA) is a search heuristic inspired by Charles Darwin's theory of natural selection. It 'evolves' a population of candidate solutions towards a better outcome over several generations.

In the context of our MPC problem, the concepts map as follows:

*   **Individual / Chromosome:** A single, complete control plan. This is a sequence of `H` future CPP values.
*   **Population:** A large collection of these control plans (e.g., 100 different individuals).
*   **Gene:** A single CPP value at a single future time step within a plan.
*   **Fitness Function:** Our MPC cost function. It evaluates a control plan (an individual) and assigns it a score (lower is better). This is the 'environment' that determines survival.

The evolutionary process works in a loop:
1.  **Selection:** The 'fittest' individuals (plans with the lowest cost) are more likely to be selected to 'reproduce'.
2.  **Crossover:** Two parent plans are combined to create a new offspring plan, mixing characteristics from both (e.g., taking the first half of Plan A and the second half of Plan B).
3.  **Mutation:** Small, random changes are introduced into the new offspring's genes. This maintains genetic diversity and helps escape local optima.

After many generations, the population converges towards a set of high-fitness individuals, one of which will be our optimal control plan.

--- 
## 3. Building the `GeneticOptimizer` Class with DEAP

We will use the powerful `DEAP` (Distributed Evolutionary Algorithms in Python) library to implement our GA. We will create a wrapper class in `src/optimizers.py` to provide a clean interface tailored to our MPC problem.

In [ ]:
%%writefile ../robust_mpc/optimizers.py
import random
import numpy as np
from deap import base, creator, tools, algorithms

class GeneticOptimizer:
    """
    A wrapper for the DEAP library to find optimal control sequences using a
    Genetic Algorithm.
    """
    def __init__(self, fitness_function, param_bounds, config):
        self.fitness_function = fitness_function
        self.param_bounds = param_bounds # List of (min, max) for each gene
        self.config = config
        self.n_params = len(param_bounds) # Number of genes in an individual

        # --- DEAP Toolbox Setup ---
        # 1. Define the fitness objective (minimizing a single value)
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        
        # 2. Define the structure of an individual (a list of floats with a fitness attribute)
        creator.create("Individual", list, fitness=creator.FitnessMin)

        self.toolbox = base.Toolbox()
        
        # 3. Gene generator: How to create a single gene (a random float within bounds)
        self.toolbox.register("attr_float", self._rand_float_in_bounds)
        
        # 4. Individual and Population generators
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attr_float, n=self.n_params)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        
        # 5. Register the evolutionary operators
        self.toolbox.register("evaluate", self._evaluate)
        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.1)
        self.toolbox.register("select", tools.selTournament, tournsize=3)

    def _rand_float_in_bounds(self):
        """Helper to create a gene by respecting the parameter bounds."""
        return [random.uniform(low, high) for low, high in self.param_bounds]
        
    def _evaluate(self, individual):
        """Wrapper to connect DEAP's evaluation to our MPC fitness function."""
        # DEAP works with a flat list, we need to reshape it into a control plan
        control_plan = np.array(individual).reshape(self.config['horizon'], self.config['num_cpps'])
        cost = self.fitness_function(control_plan)
        return (cost,) # DEAP expects a tuple
    
    def _check_bounds(self, individual):
        """Ensures that mutated/crossed-over individuals stay within bounds."""
        for i, (low, high) in enumerate(self.param_bounds):
            if individual[i] > high:
                individual[i] = high
            elif individual[i] < low:
                individual[i] = low
        return individual
        
    def optimize(self):
        """Runs the genetic algorithm to find the best control plan."""
        pop = self.toolbox.population(n=self.config['population_size'])
        
        # Use a hall of fame to keep track of the best individual found so far
        hof = tools.HallOfFame(1)
        
        # Run the evolution
        algorithms.eaSimple(
            pop,
            self.toolbox,
            cxpb=self.config['crossover_prob'],
            mutpb=self.config['mutation_prob'],
            ngen=self.config['num_generations'],
            halloffame=hof,
            verbose=False
        )
        
        # Reshape the best individual back into a plan
        best_individual = hof[0]
        best_plan = np.array(best_individual).reshape(self.config['horizon'], self.config['num_cpps'])
        
        return best_plan

--- 
## 4. Defining a Richer Action Space

The power of using a GA is that our 'chromosome' (the individual) can represent a much more complex control plan than before. In V1, we could only represent a single change held constant. Now, we will define the chromosome to be a flattened list of all CPP values at all future time steps.

**Chromosome Structure:**
`[spray_rate_t1, air_flow_t1, ..., spray_rate_t2, air_flow_t2, ..., spray_rate_tH, air_flow_tH]`

This allows the GA to discover sophisticated plans like:
*   **Ramps:** Gradually increasing the `spray_rate` over 10 steps.
*   **Step-and-Hold:** Making a sharp change in `air_flow` and then holding it.
*   **Complex Profiles:** Any arbitrary sequence of moves.

The `param_bounds` we pass to the optimizer will be a long list containing the `(min, max)` operational range for each gene in this flattened structure.

In [ ]:
def setup_ga_config_and_bounds(cpp_config, horizon):
    """Helper function to create the config and bounds list for the GA."""
    ga_config = {
        'population_size': 50,
        'num_generations': 20,
        'crossover_prob': 0.7,
        'mutation_prob': 0.2,
        'horizon': horizon,
        'num_cpps': len(cpp_config)
    }
    
    # Create the flattened list of bounds for the chromosome
    param_bounds = []
    cpp_names = list(cpp_config.keys())
    for _ in range(horizon):
        for name in cpp_names:
            param_bounds.append((cpp_config[name]['min_val'], cpp_config[name]['max_val']))
            
    return ga_config, param_bounds

# Example usage
CPP_CONSTRAINTS = {
    'spray_rate': {'min_val': 80.0, 'max_val': 180.0},
    'air_flow': {'min_val': 400.0, 'max_val': 700.0},
    'carousel_speed': {'min_val': 20.0, 'max_val': 40.0}
}
HORIZON = 10 # Use a shorter horizon for this test

ga_config, param_bounds = setup_ga_config_and_bounds(CPP_CONSTRAINTS, HORIZON)

print(f"GA Config: {ga_config}")
print(f"Length of chromosome (num_genes): {len(param_bounds)}")
print(f"Example bounds for first 3 genes: {param_bounds[:3]}")

--- 
## 5. Standalone Test of the Genetic Optimizer

Let's test our new `GeneticOptimizer`. We'll create a simple dummy 'fitness function' that we can easily understand. The goal of the fitness function will be to evolve a control plan that ramps the `spray_rate` up to its maximum value while keeping the other CPPs low.

**Dummy Fitness Goal:** `minimize(sum(air_flow) + sum(carousel_speed) - sum(spray_rate))`

In [ ]:
from robust_mpc.optimizers import GeneticOptimizer

# Define our simple dummy fitness function
def dummy_fitness_function(control_plan):
    """A simple cost function for testing the GA."""
    # control_plan shape: (horizon, num_cpps)
    spray_rate_sum = np.sum(control_plan[:, 0])
    air_flow_sum = np.sum(control_plan[:, 1])
    carousel_speed_sum = np.sum(control_plan[:, 2])
    
    # We want to minimize this value, so we penalize high air_flow/speed 
    # and reward high spray_rate (by subtracting it).
    cost = air_flow_sum + carousel_speed_sum - spray_rate_sum
    return cost

# --- Instantiate and Run the Optimizer ---
optimizer = GeneticOptimizer(dummy_fitness_function, param_bounds, ga_config)
print("Running Genetic Algorithm to find the optimal plan...")
best_plan_found = optimizer.optimize()

# --- Visualize the Result ---
df_best_plan = pd.DataFrame(best_plan_found, columns=CPP_CONSTRAINTS.keys())

print("\nGA has finished. The optimal plan found is:")
plt.figure(figsize=(15, 6))
df_best_plan.plot(ax=plt.gca())
plt.title('Control Plan Evolved by the Genetic Algorithm', fontsize=16)
plt.xlabel('Time Steps into Horizon', fontsize=12)
plt.ylabel('CPP Values', fontsize=12)
plt.legend()
plt.grid(True, linestyle=':')
plt.show()

### Final Analysis

The plot should show that the GA successfully 'discovered' a control plan that aligns with our simple fitness function. The `spray_rate` should be at or near its maximum value (180), while `air_flow` and `carousel_speed` should be at or near their minimum values (400 and 20, respectively). The shape of the lines might not be perfectly flat due to the randomness of crossover and mutation, but the overall trend will be clear.

We have now built the third major component of our V2 framework: a scalable and intelligent optimization engine. This `GeneticOptimizer` will replace the V1 controller's biggest weakness and allow our new `RobustMPCController` to find sophisticated solutions to complex control problems.